In [7]:
import numpy as np
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, ClassifierMixin
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [8]:
# Define image dimensions and batch size
img_height, img_width = 128, 128
batch_size = 32
# Define directories for training and testing data
train_data_dir = "../dataset/train"
test_data_dir = "../dataset/test"

In [9]:
# Data augmentation for training images
train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=1./255  # Normalize pixel values
)

# Data augmentation for testing images (only rescale)
test_datagen = ImageDataGenerator(rescale=1./255)


In [10]:
# Create data generators for training and testing
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # No need to shuffle for evaluation
)



Found 1174 images belonging to 14 classes.
Found 428 images belonging to 14 classes.


In [11]:
# Define the VGG16 base model
def create_vgg_model():
    vgg_model = VGG16(
        input_shape=(img_height, img_width, 3),  # Adjust input shape
        include_top=False,  # Exclude the fully-connected layers
        weights='imagenet'  # Pre-trained on ImageNet
    )
    vgg_model.trainable = False

    model = Sequential([
        vgg_model,
        GlobalAveragePooling2D(),
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        Dense(14, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    print(model.summary())
    return model




In [12]:
#MODEL fit
model = create_vgg_model()
model.fit(train_generator, epochs=15, validation_data=test_generator)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 global_average_pooling2d_1   (None, 512)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_2 (Dense)             (None, 64)                32832     
                                                                 
 batch_normalization_1 (Batc  (None, 64)               256       
 hNormalization)                                                 
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 14)               

In [13]:
#predict
predictions = model.predict(test_generator)



14/14 [==============================] - 2s 91ms/step


In [14]:
#calculate accuracy
y_pred = np.argmax(predictions, axis=1)
y_true = test_generator.classes
accuracy = accuracy_score(y_true, y_pred)
print(accuracy)

0.6308411214953271
